In [ ]:
import picamera
import time
import cv2
import tensorflow as tf
import numpy as np
import os
import sys
from glob import glob

In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 12.75 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.


In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2022-10-10 01:21:35--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  13.9MB/s    in 2.4s    

2022-10-10 01:21:38 (13.9 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [ ]:
# Yolov3-tiny model load
net = cv2.dnn.readNet("/content/drive/MyDrive/darknet-master/results/train2/lagrange_yolov3-tiny_4000.weights", "/content/drive/MyDrive/darknet-master/custom/lagrange_yolov3-tiny.cfg")
classes = []
with open("/content/drive/MyDrive/darknet-master/custom/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = ((0,255,0),(0,0,255))

In [ ]:
cap = cv2.VideoCapture(-1) # 0, 1, 2 순서로 연결된 카메라 인덱스번호, 오류발생시 -1 넣어주기

fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
#movie = cv2.VideoWriter('detection.mpeg4', fourcc, 20.0, (640,480)) # fps, size
prevTime = time.time()
while(cap.isOpened()): # 카메라 켜져있을 때 반복실행
  ret, frame = cap.read() # 프레임 하나 읽고 값 반환 (ret은 boolean, frame은 사진이 배열로 들어감)
  
  if(ret):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392,(416,416  ), (0,0,0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # confidence가 0.5 이상이면 해당 Object라고 탐지
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
      for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
          print('detected', sclasses[class_id])
          center_x = int(detection[0] * width)
          center_y = int(detection[1] * height)
          w = int(detection[2] * width)
          h = int(detection[3] * height)            
          x = int(center_x - w / 2)
          y = int(center_y - h / 2)
          boxes.append([x, y, w, h])
          confidences.append(float(confidence))
          class_ids.append(class_id)

    # 박스 중복을 방지하기 위해 noise를 지워주는 과정
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
      if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label+' '+str(confidences[i]), (x, y + 30), font, 1, color, 2)

     # 프레임 수 계산
    curTime = time.time()	# current time
    fps = 1 / (curTime - prevTime)
    prevTime = curTime
    # 프레임 수 문자열에 저장
    fps_str = "FPS : %0.1f" %fps

    # 문자열 표시
    cv2.putText(frame, fps_str, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))

    cv2.imshow('frame', frame)
    k = cv2.waitKey(1) & 0xFF
    if(k==27):
      break




cap.release()
cv2.destroyAllWindows()

Mounted at /content/drive
